In [1]:
import pandas as pd

In [2]:
# open match file
df = pd.read_csv('../match-data/nwsl-events/2025-04-13_Gotham_FC__vs__North_Carolina_Courage.csv')

In [3]:
df.head()

,id,eventId,typeId,periodId,timeMin,timeSec,contestantId,outcome,x,y,timeStamp,lastModified,qualifier,playerId,playerName,keyPass,assist,Squad,Position
0,2800998551,1,34,16,0,0,b4fbmos00gxtk8qnaqsm392p5,1,0.0,0.0,2025-04-13T19:15:13.746Z,2025-04-13T20:56:41Z,"[{'id': 5651196995, 'qualifierId': 227, 'value...",NaN,NaN,NaN,NaN,North Carolina Courage,Away
1,2801008623,1,34,16,0,0,bgkfdckqfqnfbmecbtz03n76d,1,0.0,0.0,2025-04-13T19:29:02.601Z,2025-04-13T20:54:52Z,"[{'id': 5651255981, 'qualifierId': 131, 'value...",NaN,NaN,NaN,NaN,Gotham FC,Home
2,2801039029,2,32,1,0,0,b4fbmos00gxtk8qnaqsm392p5,1,0.0,0.0,2025-04-13T20:09:10.271Z,2025-04-13T20:09:10Z,"[{'id': 5651430309, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,North Carolina Courage,Away
3,2801039039,2,32,1,0,0,bgkfdckqfqnfbmecbtz03n76d,1,0.0,0.0,2025-04-13T20:09:10.271Z,2025-04-13T20:09:12Z,"[{'id': 5651430367, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,Gotham FC,Home
4,2801039059,3,1,1,0,0,b4fbmos00gxtk8qnaqsm392p5,1,50.0,50.0,2025-04-13T20:09:10.272Z,2025-04-13T20:09:14Z,"[{'id': 5651430531, 'qualifierId': 279, 'value...",4k83z2cdvn2kwb72kh8e8qfsa,J. Shaw,NaN,NaN,North Carolina Courage,Away


In [62]:
# get shots into one dataframe- there is 'Goal' - 16, 'Attempt Saved' - 15, 'Miss' - 13, 'Post' - 14
df_shots = pd.concat([df[df.typeId == 13], df[df.typeId == 14], 
                    df[df.typeId == 15], df[df.typeId == 16]])

# get shot outcome
shot_outcomes = {13: 'Miss', 14: 'Post', 15: 'Attempt Saved', 16: 'Goal'}
df['shot_outcome'] = df_shots['typeId'].map(shot_outcomes)

# # unpack qualifiers
# for i in df_shots.index:
#     for j in df_shots['qualifier'][i]:
#         if j.get('qualifierId') == 102:
#             df_shots.at[i, 'GoalMouthX'] = float(j['value'])
#         elif j.get('qualifierId') == 103:
#             df_shots.at[i, 'GoalMouthY'] = float(j['value'])

# unpack shot qualifiers - 102 and 103, 146 and 147, 72 and 20 (left and right foot),
# 29 (assisted), 22 (open play), 154 (intentional assist), 23 (fast break), 24 (set-piece)
# 25 (corner), 26 (free-kick)
df_shots['goalMouthY'] = 0.0
df_shots['goalMouthZ'] = 0.0
for i, row in df_shots.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 102:
                df_shots.at[i, 'goalMouthY'] = float(q['value'])
            elif q.get('qualifierId') == 103:
                df_shots.at[i, 'goalMouthZ'] = float(q['value'])

df_shots['blockedX'] = 0.0
df_shots['blockedY'] = 0.0
for i, row in df_shots.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 146:
                df_shots.at[i, 'blockedX'] = float(q['value'])
            elif q.get('qualifierId') == 147:
                df_shots.at[i, 'blockedY'] = float(q['value'])

df_shots['left_foot'] = False
df_shots['right_foot'] = False
for i, row in df_shots.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 20:  # Right foot ID
                df_shots.at[i, 'right_foot'] = True
            elif q.get('qualifierId') == 72:  # Left foot ID
                df_shots.at[i, 'left_foot'] = True

df_shots['open_play'] = False
df_shots['assisted'] = False
for i, row in df_shots.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 22: 
                df_shots.at[i, 'open_play'] = True
            elif q.get('qualifierId') == 29:
                df_shots.at[i, 'assisted'] = True

# 154 (intentional assist), 23 (fast break), 24 (set-piece)
# 25 (corner), 26 (free-kick)
df['intentional_assist'] = False
df_shots['fast_break'] = False
for i, row in df_shots.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 154: 
                df_shots.at[i, 'intentional_assist'] = True
            elif q.get('qualifierId') == 23: 
                df_shots.at[i, 'fast_break'] = True

df_shots['set-piece'] = False
df_shots['corner'] = False
df_shots['free-kick'] = False
for i, row in df_shots.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 24:
                df_shots.at[i, 'set-piece'] = True
            elif q.get('qualifierId') == 25: 
                df_shots.at[i, 'corner'] = True
            elif q.get('qualifierId') == 26:  
                df_shots.at[i, 'free-kick'] = True


df_shots

,id,eventId,typeId,periodId,timeMin,timeSec,contestantId,outcome,x,y,...,blockedY,left_foot,right_foot,intentional_assist,open_play,assisted,fast_break,set-piece,corner,free-kick
10,2801039211,8,13,1,0,16,b4fbmos00gxtk8qnaqsm392p5,1,88.5,64.1,...,0.0,True,False,True,True,True,False,False,False,False
84,2801041899,50,13,1,4,18,bgkfdckqfqnfbmecbtz03n76d,1,81.0,47.9,...,0.0,False,True,False,False,False,False,False,False,False
175,2801087943,460,13,1,8,45,b4fbmos00gxtk8qnaqsm392p5,1,73.0,62.8,...,0.0,False,True,False,True,True,False,False,False,False
195,2801045701,85,13,1,9,32,b4fbmos00gxtk8qnaqsm392p5,1,79.3,37.8,...,0.0,False,True,False,True,False,False,False,False,False
1078,2801092251,548,13,2,55,12,bgkfdckqfqnfbmecbtz03n76d,1,92.8,44.2,...,0.0,False,True,False,True,True,False,False,False,False
1291,2801097749,650,13,2,67,21,bgkfdckqfqnfbmecbtz03n76d,1,77.7,60.2,...,0.0,True,False,False,True,False,False,False,False,False
1607,2801105781,808,13,2,87,52,bgkfdckqfqnfbmecbtz03n76d,1,88.6,74.4,...,0.0,False,True,False,True,False,False,False,False,False
1695,2801107715,848,13,2,92,45,bgkfdckqfqnfbmecbtz03n76d,1,93.8,37.1,...,0.0,False,True,False,True,True,False,False,False,False
470,2801055681,231,14,1,23,39,b4fbmos00gxtk8qnaqsm392p5,1,78.3,53.4,...,0.0,False,True,False,True,False,False,False,False,False
1134,2801093431,575,14,2,58,1,bgkfdckqfqnfbmecbtz03n76d,1,81.1,44.3,...,0.0,False,True,False,True,True,False,False,False,False


In [56]:
print(df['keyPass'].notna().sum())

15


In [66]:
df_shots.columns

Index(['id', 'eventId', 'typeId', 'periodId', 'timeMin', 'timeSec',
       'contestantId', 'outcome', 'x', 'y', 'timeStamp', 'lastModified',
       'qualifier', 'playerId', 'playerName', 'keyPass', 'assist', 'Squad',
       'Position', 'goalMouthY', 'goalMouthZ', 'blockedX', 'blockedY',
       'left_foot', 'right_foot', 'intentional_assist', 'open_play',
       'assisted', 'fast_break', 'set-piece', 'corner', 'free-kick'],
      dtype='object')

In [60]:
df_shots.drop(columns=['lastModified', 'timeStamp', 'qualifier'])

,id,eventId,typeId,periodId,timeMin,timeSec,contestantId,outcome,x,y,...,blockedY,left_foot,right_foot,intentional_assist,open_play,assisted,fast_break,set-piece,corner,free-kick
10,2801039211,8,13,1,0,16,b4fbmos00gxtk8qnaqsm392p5,1,88.5,64.1,...,0.0,True,False,True,True,True,False,False,False,False
84,2801041899,50,13,1,4,18,bgkfdckqfqnfbmecbtz03n76d,1,81.0,47.9,...,0.0,False,True,False,False,False,False,False,False,False
175,2801087943,460,13,1,8,45,b4fbmos00gxtk8qnaqsm392p5,1,73.0,62.8,...,0.0,False,True,False,True,True,False,False,False,False
195,2801045701,85,13,1,9,32,b4fbmos00gxtk8qnaqsm392p5,1,79.3,37.8,...,0.0,False,True,False,True,False,False,False,False,False
1078,2801092251,548,13,2,55,12,bgkfdckqfqnfbmecbtz03n76d,1,92.8,44.2,...,0.0,False,True,False,True,True,False,False,False,False
1291,2801097749,650,13,2,67,21,bgkfdckqfqnfbmecbtz03n76d,1,77.7,60.2,...,0.0,True,False,False,True,False,False,False,False,False
1607,2801105781,808,13,2,87,52,bgkfdckqfqnfbmecbtz03n76d,1,88.6,74.4,...,0.0,False,True,False,True,False,False,False,False,False
1695,2801107715,848,13,2,92,45,bgkfdckqfqnfbmecbtz03n76d,1,93.8,37.1,...,0.0,False,True,False,True,True,False,False,False,False
470,2801055681,231,14,1,23,39,b4fbmos00gxtk8qnaqsm392p5,1,78.3,53.4,...,0.0,False,True,False,True,False,False,False,False,False
1134,2801093431,575,14,2,58,1,bgkfdckqfqnfbmecbtz03n76d,1,81.1,44.3,...,0.0,False,True,False,True,True,False,False,False,False
